In [15]:
#By Ricardo Kazuo
import os
import csv
import time
import datetime
from datetime import datetime, timedelta, timezone
import requests
import polars as pl
import numpy as np
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup as bs
from pandas_datareader import data

def fetch_on_stock(ticker_in,country):
    """By Ricardo Kazuo"""
    #Fetches all the historically available values for one ticker, received as parameter using yfinance
    #File parameter is where the values will be stored
    try:
        #Let's use yfinance as yf to fetch the data
        ticker= yf.Ticker(ticker_in)
        print(ticker_in)
        historical = ticker.history(start="1900-01-01", end=datetime.now(), interval="1d")
        #Let's drop unwanted columns
        print(historical)
        historical.drop(columns=['High', 'Low', 'Volume', 'Dividends', 'Stock Splits'],inplace=True)
        #Let's us add a column with the ticker name
        historical.insert(0, "Ticker", ticker_in, True)
        #Let's reset the index
        historical.reset_index(inplace=True)
        #Let's format the date to a propepr format
        historical['Date']=historical['Date'].dt.tz_localize(None)
        historical = historical.sort_values(by="Date")
        historical = historical[historical["Close"] != "Close"]
        historical["Log_Python"] = np.log2(historical["Close"])
        historical["Log_Diff"] = historical["Log_Python"].diff(1)
        try:
            historical.drop(columns=['Capital Gains'],inplace=True)
        except:
            pass
        historical = historical.dropna()
        print(historical)
        #Let's save to a csv file
        historical.to_csv(country+".csv", mode='a',index=False, header=False)
    except:
        pass


def fetch_data_watchlist():
    """By Ricardo Kazuo"""
#Inputs:WatchlistTicker.csv
#Inputs:Symbol.csv
#Outputs: Symbol.csv
    try:
        #Let's read the first csv file
        df_watchlist = pd.read_csv(('WatchlistTicker.csv'),on_bad_lines='skip')
        #Let's select the column "Symbol"
        df_watchlist = df_watchlist['Symbol']
        #Let's read the second csv file
        df_symbol = pd.read_csv(('Symbol.csv'),on_bad_lines='skip')
        #Let's select the column "Symbol"
        df_symbol = df_symbol['Symbol']
        frames = [df_watchlist, df_symbol]
        #Let's concatenate both dataframes
        df_result = pd.concat(frames, sort=True)
        #Let's drop the duplicate entries
        df_result = df_result.drop_duplicates()
        #Let's sort the entries
        df_result = df_result.sort_values()
        #Let's save to a file
        df_result.to_csv('Symbol.csv', mode='w',index=False, header=True)
    except:
        pass

def str_trim(x_in):
    """By Ricardo Kazuo"""
    try:
        if len(x_in) == 2:
            return x_in[:1]
        elif len(x_in) == 4:
            return x_in[:2]
        elif len(x_in) == 6:
            return x_in[:3]
        elif len(x_in) == 8:
            return x_in[:4]
        elif len(x_in) == 10:
            return x_in[:5]
        elif len(x_in) == 12:
            return x_in[:6]
        elif len(x_in) == 14:
            return x_in[:7]
        elif len(x_in) == 16:
            return x_in[:8]
        elif len(x_in) == 18:
            return x_in[:9]
        else:
            return x_in
    except:
        return x_in

def fetch_watchlists():
    """By Ricardo Kazuo"""
#Fetches all the ticker by category
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'}
    data = pd.DataFrame()
    address = {
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/video-game-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/420_stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/tech-stocks-that-move-the-market",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/new-family-economy",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/sinful-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/romance-industry",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/bank-and-financial-services-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/cash-rich-companies-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/biotech-and-drug-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/semiconductor-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/oil-and-gas-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/china-tech-and-internet-stocks",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/most-watched",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/women-at-the-helm",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/healthy-living",
    "https://finance.yahoo.com/u/yahoo-finance/watchlists/electronic-trading-stocks",
    "https://finance.yahoo.com/u/trea/watchlists/the-fight-against-covid19",
    "https://finance.yahoo.com/u/motley-fool/watchlists/dividend-growth-market-leaders",
    "https://finance.yahoo.com/u/motley-fool/watchlists/8-buffett-buys",
    "https://finance.yahoo.com/u/motley-fool/watchlists/brands-consumers-love",
    "https://finance.yahoo.com/u/motley-fool/watchlists/stocks-fueling-internet-of-things",
    "https://finance.yahoo.com/world-indices",
    "https://finance.yahoo.com/commodities",
	"https://finance.yahoo.com/currencies"
    }
    address1 = {
    "https://finance.yahoo.com/screener/predefined/insurance_life",
    "https://finance.yahoo.com/screener/predefined/aerospace_defense",
    "https://finance.yahoo.com/screener/predefined/auto_manufacturers",
    "https://finance.yahoo.com/screener/predefined/growth_technology_stocks",
    "https://finance.yahoo.com/screener/predefined/most_shorted_stocks",
    "https://finance.yahoo.com/losers",
    "https://finance.yahoo.com/gainers",
    "https://finance.yahoo.com/mutualfunds"
    }
    for addres in address:
        try:
            file = addres.partition('s/')[2] or \
                    addres.partition('d/')[2] or \
                    addres.partition('m/')[2]
            print(file)
            temp = requests.get(addres,headers=headers).text
            soup = bs(temp, 'html.parser')
            table=soup.findAll("table", {"class": "cwl-symbols"})
            if len(table)==0 :
                table=soup.findAll("table", {"class": "W(100%)"})
            try:
                temp =pd.read_html(str(table))[0]
                temp.insert(1, "Type", file, True)
                temp = temp.iloc[:, [0,1]]
                data = pd.concat([data,temp], ignore_index=True, sort=False)
            except:
                pass
        except:
            pass
    for addres in address1:
        try:
            file = addres.partition('s/')[2] or \
                    addres.partition('d/')[2] or \
                    addres.partition('m/')[2]
            print(file)
            temp = requests.get(addres,headers=headers).text
            soup = bs(temp, 'html.parser')
            table=soup.findAll("table", {"class": "cwl-symbols"})
            if len(table)==0 :
                table=soup.findAll("table", {"class": "W(100%)"})
            try:
                temp=pd.read_html(str(table))[0]
                temp.insert(1, "Type", file, True)
                temp.iloc[:,0]=temp.iloc[:,0].str.replace(r'Select ', '')
                temp.iloc[:,0]=temp.iloc[:,0].apply(lambda x: str_trim(x))
                temp.rename(columns={ temp.columns[0]: "Symbol" }, inplace = True)
                temp=temp.iloc[:,[0,1]]
                data=pd.concat([data,temp], ignore_index=True, sort=False)
            except:
                pass
        except:
            pass
    data.iloc[:, :3].to_csv("WatchlistTicker.csv",mode='w',index=False,header=True)
    data = pd.DataFrame()

def finance_db(country_in):
    #print(f"{country_in}-{append}-{path}")
    df_symbol = pd.read_csv("FinancialData.csv",on_bad_lines='skip')
    df_symbol = df_symbol[df_symbol['country'] == country_in]
    count = len(df_symbol.index)
    for index,row in df_symbol.iterrows():
        fetch_on_stock(row['symbol'],country_in)
        count = count - 1

def clear_file(file_in):
    """By Ricardo Kazuo"""
    file_to_delete = open(file_in,'w')
    file_to_delete.close() 

def main():
    #fetch_watchlists()
    #fetch_data_watchlist()
    clear_file("Brazil.csv")
    finance_db("Brazil")


if __name__ == '__main__':
    main()


ABEV
                               Open      High       Low     Close    Volume  \
Date                                                                          
1997-03-05 00:00:00-05:00  0.000000  0.208569  0.208569  0.208569     15000   
1997-03-06 00:00:00-05:00  0.208569  0.208569  0.208569  0.208569         0   
1997-03-07 00:00:00-05:00  0.208569  0.208569  0.208569  0.208569         0   
1997-03-10 00:00:00-05:00  0.000000  0.225950  0.225950  0.225950     24000   
1997-03-11 00:00:00-05:00  0.225950  0.225950  0.225950  0.225950         0   
...                             ...       ...       ...       ...       ...   
2023-06-12 00:00:00-04:00  3.070000  3.100000  3.030000  3.080000  17659300   
2023-06-13 00:00:00-04:00  3.090000  3.130000  3.070000  3.100000  13213900   
2023-06-14 00:00:00-04:00  3.120000  3.160000  3.070000  3.140000  17622200   
2023-06-15 00:00:00-04:00  3.140000  3.160000  3.090000  3.130000  15677100   
2023-06-16 00:00:00-04:00  3.090000  3.150000  

                                Open       High        Low      Close  \
Date                                                                    
2017-04-11 00:00:00-04:00  22.000000  22.100000  21.250000  21.500000   
2017-04-12 00:00:00-04:00  21.420000  22.030001  21.330000  21.940001   
2017-04-13 00:00:00-04:00  22.030001  22.180000  21.674999  21.879999   
2017-04-17 00:00:00-04:00  22.049999  22.190001  21.850000  22.059999   
2017-04-18 00:00:00-04:00  22.180000  23.950001  22.059999  23.100000   
...                              ...        ...        ...        ...   
2023-06-12 00:00:00-04:00  12.110000  12.190000  11.655000  11.880000   
2023-06-13 00:00:00-04:00  11.910000  11.920000  11.200000  11.290000   
2023-06-14 00:00:00-04:00  11.640000  12.370000  11.545000  12.350000   
2023-06-15 00:00:00-04:00  12.500000  12.660000  12.290000  12.500000   
2023-06-16 00:00:00-04:00  12.440000  12.930000  12.430000  12.770000   

                             Volume  Dividends  St

                               Open      High       Low     Close   Volume  \
Date                                                                         
2014-03-31 00:00:00-04:00  6.005347  6.070564  6.005347  6.005347  1399900   
2014-04-01 00:00:00-04:00  6.065129  6.141215  6.043390  6.135780   341000   
2014-04-02 00:00:00-04:00  6.195562  6.195562  6.130345  6.141215    30000   
2014-04-03 00:00:00-04:00  6.108606  6.114041  6.108606  6.114041    71700   
2014-04-04 00:00:00-04:00  6.065129  6.168388  6.059694  6.135780   356700   
...                             ...       ...       ...       ...      ...   
2023-06-12 00:00:00-04:00  6.490000  6.490000  6.370000  6.370000    40700   
2023-06-13 00:00:00-04:00  6.350000  6.380000  6.200000  6.280000   845500   
2023-06-14 00:00:00-04:00  6.290000  6.390000  6.260000  6.260000    22400   
2023-06-15 00:00:00-04:00  6.400000  6.480000  6.400000  6.440000    53200   
2023-06-16 00:00:00-04:00  6.390000  6.450000  6.330000  6.33000

Got error from yahoo api for ticker BMIX, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- BMIX: No timezone found, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
BOLSY
                               Open      High       Low     Close  Volume  \
Date                                                                        
2021-04-12 00:00:00-04:00  9.190311  9.190311  9.190311  9.190311     100   
2021-04-13 00:00:00-04:00  9.190311  9.190311  9.190311  9.190311       0   
2021-04-14 00:00:00-04:00  9.190311  9.190311  9.190311  9.190311       0   
2021-04-15 00:00:00-04:00  9.190311  9.190311  9.190311  9.190311       0   
2021-04-16 00:00:00-04:00  9.190311  9.190311  9.190311  9.190311       0   
...                             ...       ...       ...       ...     ...   
2023-06-12 00:00:00-04:00  8.050000  8.050000  8.050000  8.050000       0   
2023-06-13 00:00:00-04:00  8.050000  8.050000

Got error from yahoo api for ticker BTOOY, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- BTOOY: No timezone found, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
CBD
                               Open      High       Low     Close  Volume  \
Date                                                                        
1996-03-08 00:00:00-05:00  4.521682  4.652115  4.521682  4.652115   54000   
1996-03-11 00:00:00-05:00  4.782547  4.782547  4.782547  4.782547     400   
1996-03-12 00:00:00-05:00  4.782547  4.782547  4.782547  4.782547       0   
1996-03-13 00:00:00-05:00  4.739070  4.739070  4.739070  4.739070   10000   
1996-03-14 00:00:00-05:00  4.782547  4.782547  4.782547  4.782547   10200   
...                             ...       ...       ...       ...     ...   
2023-06-12 00:00:00-04:00  3.520000  3.580000  3.450000  3.550000  432700   
2023-06-13 00:00:00-04:00  3.590000  3.610000

...

                               Open      High       Low     Close  Volume  \
Date                                                                        
2012-04-18 00:00:00-04:00  3.187899  3.187899  3.187899  3.187899     400   
2012-04-19 00:00:00-04:00  3.187899  3.187899  3.187899  3.187899       0   
2012-04-20 00:00:00-04:00  3.187899  3.187899  3.187899  3.187899       0   
2012-04-23 00:00:00-04:00  3.251657  3.251657  3.251657  3.251657    3600   
2012-04-24 00:00:00-04:00  3.251657  3.251657  3.251657  3.251657       0   
...                             ...       ...       ...       ...     ...   
2023-06-12 00:00:00-04:00  6.950000  7.220000  6.900000  7.220000  168100   
2023-06-13 00:00:00-04:00  7.200000  7.460000  7.100000  7.100000    8500   
2023-06-14 00:00:00-04:00  7.100000  7.100000  7.100000  7.100000   18100   
2023-06-15 00:00:00-04:00  6.980000  7.210000  6.980000  7.110000   36800   
2023-06-16 00:00:00-04:00  7.410000  7.420000  7.410000  7.410000  162400   

                                Open       High        Low      Close  Volume  \
Date                                                                            
2010-01-05 00:00:00-05:00  11.805425  11.975454  11.708817  11.708817   26600   
2010-01-06 00:00:00-05:00  11.770648  11.902034  11.554248  11.824749   23600   
2010-01-07 00:00:00-05:00  11.399674  11.616074  11.399674  11.534925   47800   
2010-01-08 00:00:00-05:00  11.670175  11.766782  11.496281  11.689496   49600   
2010-01-11 00:00:00-05:00  11.805425  11.805425  11.554246  11.786104   46400   
...                              ...        ...        ...        ...     ...   
2023-06-12 00:00:00-04:00   1.420000   1.490000   1.420000   1.490000   10900   
2023-06-13 00:00:00-04:00   1.490000   1.490000   1.470000   1.480000    9600   
2023-06-14 00:00:00-04:00   1.440000   1.500000   1.380000   1.500000   41600   
2023-06-15 00:00:00-04:00   1.450000   1.520000   1.450000   1.510000   29600   
2023-06-16 00:00:00-04:00   

                                Open       High    Low      Close    Volume  \
Date                                                                          
2020-07-31 00:00:00-04:00  22.100000  22.350000  16.77  18.850000  11079900   
2020-08-03 00:00:00-04:00  18.540001  19.430000  18.00  19.000000    627600   
2020-08-04 00:00:00-04:00  19.000000  19.000000  17.76  18.020000    864900   
2020-08-05 00:00:00-04:00  18.020000  18.434999  17.42  17.500000   1755000   
2020-08-06 00:00:00-04:00  17.500000  18.330000  17.50  17.530001    528300   
...                              ...        ...    ...        ...       ...   
2023-06-12 00:00:00-04:00   3.300000   3.345000   3.26   3.280000     48100   
2023-06-13 00:00:00-04:00   3.260000   3.320000   3.26   3.320000     62400   
2023-06-14 00:00:00-04:00   3.350000   3.430000   3.27   3.300000    142700   
2023-06-15 00:00:00-04:00   3.320000   3.400000   3.31   3.390000    484500   
2023-06-16 00:00:00-04:00   3.410000   3.800000   3.

C:\Users\ricar\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


                                Open       High        Low      Close  \
Date                                                                    
2019-12-11 00:00:00-05:00  32.750000  35.000000  32.099998  34.459999   
2019-12-12 00:00:00-05:00  34.080002  37.459999  33.430000  34.639999   
2019-12-13 00:00:00-05:00  34.700001  38.320000  34.099998  37.709999   
2019-12-16 00:00:00-05:00  37.820000  38.490002  34.750000  35.799999   
2019-12-17 00:00:00-05:00  35.860001  35.860001  33.363998  34.299999   
...                              ...        ...        ...        ...   
2023-06-12 00:00:00-04:00  20.650000  21.325001  20.530001  21.190001   
2023-06-13 00:00:00-04:00  21.330000  21.389999  20.370001  20.420000   
2023-06-14 00:00:00-04:00  20.500000  21.219999  20.475000  21.139999   
2023-06-15 00:00:00-04:00  21.139999  21.875000  21.129999  21.860001   
2023-06-16 00:00:00-04:00  21.570000  22.570000  21.520000  22.129999   

                             Volume  Dividends  St